In [3]:
import os

In [4]:
os.getcwd()

'C:\\Users\\DIKSHANT PATEL\\Kidney-Disease-Classification\\research'

In [5]:
os.chdir('./..')

In [6]:
%pwd

'C:\\Users\\DIKSHANT PATEL\\Kidney-Disease-Classification'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class OptunaConfig:
      min_n_conv_layers: int
      max_n_conv_layers: int
      min_n_dense_layers: int
      max_n_dense_layers: int
      optimizer: list
      Conv2D_strides_size: list
      MaxPooling2D_strides_size: list
      filters: list
      dense_units: list
      Conv2D_kernel_size: list
      MaxPooling2D_kernel_size: list
      activation: list
      metrics: list
      epochs: int
      loss: str
      

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
    def get_Optuna_config(self) -> OptunaConfig:
        
        Optuna_params = self.params.Optuna_tuning
        
        return OptunaConfig(
            min_n_conv_layers = Optuna_params.min_n_conv_layers,
            max_n_conv_layers= Optuna_params.max_n_conv_layers,
            min_n_dense_layers= Optuna_params.min_n_dense_layers,
            max_n_dense_layers= Optuna_params.max_n_dense_layers,
            optimizer= Optuna_params.optimizer,
            Conv2D_strides_size= Optuna_params.Conv2D_strides_size,
            MaxPooling2D_strides_size= Optuna_params.MaxPooling2D_strides_size,
            filters= Optuna_params.filters,
            dense_units= Optuna_params.dense_units,
            Conv2D_kernel_size= Optuna_params.Conv2D_kernel_size,
            MaxPooling2D_kernel_size = Optuna_params.MaxPooling2D_kernel_size,
            activation= Optuna_params.activation,
            metrics= Optuna_params.metrics,
            epochs= Optuna_params.epochs,
            loss= Optuna_params.loss,
        
        )
        

In [10]:
!pip install optuna-integration

You should consider upgrading via the 'C:\Users\DIKSHANT PATEL\Kidney-Disease-Classification\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential,Model,load_model
from tensorflow.keras.layers import(
Conv2D, MaxPooling2D, Dense, Dropout,BatchNormalization, GlobalAveragePooling2D)

from tensorflow.keras.callbacks import (EarlyStopping, ModelCheckpoint, ReduceLROnPlateau)
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
import optuna

import mlflow
import mlflow.tensorflow

from cnnClassifier import logger
from mlflow.models import infer_signature
import dagshub


In [12]:

from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import gc
from tensorflow.keras import backend as K

In [13]:
from optuna.samplers import TPESampler

In [17]:
class OptunaModelTunner11:
    def __init__(self,params: OptunaConfig, training_set, validation_set, train_df, callbacks, class_weights_dict, class_names):
        self.params = params
        self.training_set = training_set
        self.validation_set = validation_set
        self.train_df = train_df
        self.callbacks = callbacks
        self.class_weights_dict = class_weights_dict
        self.class_names = class_names
        
    def create_model(self, trial):
        n_conv_layers = trial.suggest_int("n_conv_layers", self.params.min_n_conv_layers, self.params.max_n_conv_layers)
        n_dense_layers = trial.suggest_int("n_dense_layers", self.params.min_n_dense_layers, self.params.max_n_dense_layers)
        optimizer_name = trial.suggest_categorical('optimizer', self.params.optimizer)
        Conv2D_strides_size = trial.suggest_categorical("Conv2D_strides_size",self.params.Conv2D_strides_size) 
        Conv2D_stride = tuple(map(int, Conv2D_strides_size.lower().split("x")))
        
        Conv2D_kernel_size_str = trial.suggest_categorical("Conv2D_kernel_size", self.params.Conv2D_kernel_size)
        Conv2D_kernel_size = tuple(map(int, Conv2D_kernel_size_str.lower().split("x")))
        
        
        MaxPooling2D_strides_size = trial.suggest_categorical("MaxPooling2D_strides_size",self.params.MaxPooling2D_strides_size) 
        MaxPooling2D_stride = tuple(map(int, MaxPooling2D_strides_size.lower().split("x")))
        
        MaxPooling2D_kernel_size_str = trial.suggest_categorical("MaxPooling2D_kernel_size", self.params.MaxPooling2D_kernel_size)
        MaxPooling2D_kernel_size = tuple(map(int, MaxPooling2D_kernel_size_str.lower().split("x")))
        
     
        lr = trial.suggest_float("learning_rate", 0.00001, 0.001, log=True)
        filter_0 = 32
        model = Sequential()
        model.add(Conv2D(filter_0, Conv2D_kernel_size, strides=Conv2D_stride, activation="relu", padding='same', input_shape=(224,224,1), kernel_initializer='he_normal'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(MaxPooling2D_kernel_size, strides=MaxPooling2D_stride, padding='same'))
        
        for i in range(n_conv_layers):
            filters = trial.suggest_categorical(f"filters{i}", self.params.filters)
            model.add(Conv2D(filters, Conv2D_kernel_size, strides=Conv2D_stride, activation="relu", padding='same'))
            model.add(BatchNormalization())
            model.add(MaxPooling2D(MaxPooling2D_kernel_size, strides=MaxPooling2D_stride, padding='same'))
            
        model.add(GlobalAveragePooling2D())
        
        for i in range(n_dense_layers):
            dense_units = trial.suggest_categorical(f"dense_units{i}",self.params.dense_units)
            model.add(Dense(dense_units, activation='relu'))
            model.add(BatchNormalization())
            model.add(Dropout(0.3))
        
        model.add(Dense(4, activation='softmax'))
        # 
        # Allfilters = []
        # 
        # # Add filter_0 manually if needed
        # filter_0 = 32
        # Allfilters.append(filter_0)
        # 
        # if n_conv_layers == 6:
        #    filter6_1 = trial.suggest_categorical("filter6_1", [32, 64])
        #    filter6_2= 64
        #    filter6_3= trial.suggest_categorical("filter6_3", [64, 128])
        #    filter6_4= 128
        #    filter6_5 = 512
        #    Allfilters += [filter6_1, filter6_2, filter6_3, filter6_4, filter6_5]
        # 
        # elif n_conv_layers == 5:
        #    filter5_1 = trial.suggest_categorical("filter5_1", [32, 64])
        #    filter5_2 = trial.suggest_categorical("filter5_2", [64, 128])
        #    filter5_3 = trial.suggest_categorical("filter5_3", [128, 512])
        #    filter5_4 = 512
        #    Allfilters += [filter5_1, filter5_2, filter5_3, filter5_4]
        # 
        # elif n_conv_layers == 4:
        #   filter4_1 = trial.suggest_categorical("filter4_1", [32, 64])
        #   filter4_2 = 64
        #   filter4_3 = trial.suggest_categorical("filter4_3", [128, 512])
        #   Allfilters += [filter4_1, filter4_2, filter4_3]
        # 
        # else:
        #   filter3_1 = trial.suggest_categorical("filter3_1", [32, 64])
        #   filter3_2 = trial.suggest_categorical("filter3_2", [64, 128])
        #   Allfilters += [filter3_1, filter3_2]
        # 
        # 
        # model = Sequential()
        # 
        # 
        #  # First layer with input shape
        # model.add(Conv2D(Allfilters[0],Conv2D_kernel_size, strides=Conv2D_stride, activation='relu', padding='same', input_shape=(224, 224, 1)))
        # model.add(BatchNormalization())
        # model.add(MaxPooling2D(MaxPooling2D_kernel_size, strides=MaxPooling2D_stride, padding='same'))
        # 
        # # Remaining conv layers
        # for i in range(1, n_conv_layers):
        #      model.add(Conv2D(Allfilters[i],Conv2D_kernel_size, strides=Conv2D_stride, activation='relu', padding='same'))
        #      model.add(BatchNormalization())
        #      model.add(MaxPooling2D(MaxPooling2D_kernel_size, strides=MaxPooling2D_stride, padding='same'))
        #             
        # model.add(GlobalAveragePooling2D()) 
        # 
        # 
        # 
        # Alldenseunits = []
        # 
        # if n_dense_layers == 5:
        #    dense_units5_1 = trial.suggest_categorical("dense_units5_1", [256, 128])
        #    dense_units5_2 = trial.suggest_categorical("dense_units5_2", [128, 64])
        #    dense_units5_3 = trial.suggest_categorical("dense_units5_3", [64, 32])
        #    dense_units5_4 = trial.suggest_categorical("dense_units5_4", [32, 10])
        #    Alldenseunits = [dense_units5_1, dense_units5_2, dense_units5_3, dense_units5_4]
        # 
        # elif n_dense_layers == 4:
        #    dense_units4_1 = trial.suggest_categorical("dense_units4_1", [256, 128])
        #    dense_units4_2 = trial.suggest_categorical("dense_units4_2", [128, 64])
        #    dense_units4_3 = trial.suggest_categorical("dense_units4_3", [64, 32])
        #    Alldenseunits = [dense_units4_1, dense_units4_2, dense_units4_3]
        # 
        # elif n_dense_layers == 3:
        #    dense_units3_1 = trial.suggest_categorical("dense_units3_1", [128, 64])
        #    dense_units3_2 = trial.suggest_categorical("dense_units3_2", [64, 32])
        #    Alldenseunits = [dense_units3_1, dense_units3_2]
        # 
        # else:
        #    dense_units2_1 = trial.suggest_categorical("dense_units2_1", [32, 64])
        #    Alldenseunits = [dense_units2_1]
        # 
        # for units in Alldenseunits:
        #    model.add(Dense(units, activation='relu'))
        #    model.add(BatchNormalization())
        #    model.add(Dropout(0.3))
        # 
        # model.add(Dense(4, activation='softmax')) 


            # Optimizer
        if optimizer_name == 'adam':
            optimizer = Adam(learning_rate=lr)
        elif optimizer_name == 'sgd':
            optimizer = SGD(learning_rate=lr, momentum= 0.9, nesterov=True)
        elif optimizer_name == "RMSprop":
            optimizer = RMSprop(learning_rate=lr, rho=0.9, momentum=0.0, epsilon=1e-07, clipvalue=1.0)
        else:
             raise ValueError("Choose 'adam' ,'sgd' or 'RMSprop' for optimizer_name")

        model.compile(optimizer=optimizer,
                  loss=self.params.loss,
                  metrics=self.params.metrics)

        return model
    
    def objective(self, trial):
        dagshub.init(repo_owner='DikshantPatel2210', repo_name='KidneyDiseaseClassification-DLProject', mlflow=True)
        mlflow.set_tracking_uri("https://dagshub.com/DikshantPatel2210/KidneyDiseaseClassification-DLProject.mlflow")
        mlflow.set_experiment("CNN_Optuna_MLFLOW")
        try:
            if mlflow.active_run():
               mlflow.end_run()

            input_example = np.random.rand(32, 224, 224, 1).astype(float)
            output_example = np.random.rand(32, 4).astype(float)

            with mlflow.start_run(run_name=f"trial_{trial.number}"):
                n_conv_layers = trial.suggest_int("n_conv_layers", 3, 6)
                n_dense_layers = trial.suggest_int("n_dense_layers", 2, 5)
                model = self.create_model(trial)

                # Log hyperparameters
                for param_name, param_value in trial.params.items():
                    mlflow.log_param(param_name, param_value)

                history = model.fit(
                    self.training_set,
                    steps_per_epoch=len(self.train_df) // 32,
                    validation_data=self.validation_set,
                    epochs=self.params.epochs,
                    #callbacks =[tf.keras.callbacks.EarlyStopping(patience=20, min_delta=0.001, baseline=0.99,  
                    #                                             monitor="val_accuracy", mode="max", verbose=1, restore_best_weights=True),
                    #           tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=3, min_lr=1e-6, cooldown=2, 
                    #                                                 min_delta=0.001, monitor="val_loss", mode="min", verbose=1),
                    #            tf.keras.callbacks.ModelCheckpoint(monitor="val_accuracy", mode="max", 
                    #                                               filepath="artifacts/checkpoints/best_model.keras", save_best_only=True, verbose=1, save_weights_only=False)],
                    callbacks=self.callbacks,
                    class_weight=self.class_weights_dict,
                    verbose=1
                )

                # Static params
                mlflow.log_param("image_size", "224x224x1")
                mlflow.log_param("filter_0", 32)
                mlflow.log_param("last_dense_units", 4)
                mlflow.log_param("Batchsize", 32)
                mlflow.log_param("loss function", "categorical_crossentropy")
                
                if n_conv_layers == 6:
                    mlflow.log_param("filter6_2", 64)
                    mlflow.log_param("filter6_4", 128)
                    mlflow.log_param("filter6_5", 512)
                if n_conv_layers == 5:
                    mlflow.log_param("filter5_4", 512)
                if n_conv_layers == 4:
                    mlflow.log_param("filter4_2", 64)
                 
                mlflow.log_param(f"dense_units{n_dense_layers}_{n_dense_layers}", 4)   
            
            
                if np.any(np.isnan(history.history['loss'])) or np.any(np.isnan(history.history['val_loss'])):
                    raise ValueError("NaN value encountered in loss or validation loss.")
                
                
                # Log the metrics (train and validation accuracy, loss)
                train_accuracy = max(history.history['accuracy'])  # or 'acc', depending on your Keras version
                train_loss = min(history.history['loss'])
                val_accuracy = max(history.history['val_accuracy'])
                val_loss = min(history.history['val_loss'])
                
                loss_train = history.history['loss'][-1]
                loss_val = history.history['val_loss'][-1]
                acc_val = history.history['val_accuracy'][-1]
                loss_diff = abs(loss_train - loss_val)
                objective_value = acc_val - loss_diff
           # Log metrics to MLflow
                mlflow.log_metric("train_accuracy", train_accuracy)
                mlflow.log_metric("train_loss", train_loss)
                mlflow.log_metric("val_accuracy", val_accuracy)
                mlflow.log_metric("val_loss", val_loss)
                mlflow.log_metric("acc_val - loss_diff", objective_value)

                # Log model
                signature = infer_signature(input_example, output_example)
                mlflow.tensorflow.log_model(model, artifact_path="model", signature=signature)

                return objective_value

        except Exception as e:
            print(f"[Trial Failed] Error: {e}")
            mlflow.log_param("failed_trial", True)
            mlflow.log_param("error_msg", str(e)[:500])
            return float("nan")

        finally:
            mlflow.end_run()
            try:
                del model
            except:
                pass
            K.clear_session()
            gc.collect()

In [20]:
from cnnClassifier.config.configuration import ConfigurationManager
from cnnClassifier.components.data_loader import DataLoader
from cnnClassifier.components.callback import CallbackHandler


import optuna

try:
    config = ConfigurationManager()

    # Load data
    data_loader_config = config.get_data_loader_config()
    data_loader = DataLoader(config=data_loader_config)
    train_generator, val_generator, test_generator,train_df, ori_train = data_loader.get_generators()

    # Callbacks
    callbacks_config = config.get_callbacks_config()
    handler = CallbackHandler(config=callbacks_config, ori_training_set=ori_train)
    class_weights = handler.get_class_weights()
    callbacks = handler.get_callbacks()

    # Optuna Config
    optuna_config = config.get_Optuna_config()

    # Optuna Tuner
    optuna_tunner = OptunaModelTunner11(
        params=optuna_config,
        training_set=train_generator,
        validation_set=val_generator,
        train_df=train_df,
        callbacks=callbacks,
        class_weights_dict=class_weights,
        class_names=list(ori_train.class_indices.keys())
    )

    # Run Optuna Study
    study = optuna.create_study(direction="maximize",
                                sampler=optuna.samplers.TPESampler(),
                                study_name="kidney_optuna_study",
                                storage="sqlite:///optuna_study.db",  # this saves to file
                                load_if_exists=True )
    study.optimize(optuna_tunner.objective, n_trials=30)

    

except Exception as e:
    print(f"An error occurred: {e}")
    raise e



[2025-05-13 11:26:02,729: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-13 11:26:02,745: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-13 11:26:02,746: INFO: common: created directory at: artifacts]
[2025-05-13 11:26:02,747: INFO: data_loader: Loading dataframe from file: artifacts/data_split/train.csv]
[2025-05-13 11:26:02,772: INFO: data_loader: Dataframe loaded successfully with 8712 records.]
[2025-05-13 11:26:02,773: INFO: data_loader: Loading dataframe from file: artifacts/data_split/val.csv]
[2025-05-13 11:26:02,781: INFO: data_loader: Dataframe loaded successfully with 1121 records.]
[2025-05-13 11:26:02,782: INFO: data_loader: Loading dataframe from file: artifacts/data_split/test.csv]
[2025-05-13 11:26:02,792: INFO: data_loader: Dataframe loaded successfully with 2613 records.]
[2025-05-13 11:26:03,333: INFO: data_loader: Valid file paths count: 8712]
[2025-05-13 11:26:03,457: INFO: data_loader: Valid file paths count: 1121]
[2

[I 2025-05-13 11:26:05,230] A new study created in memory with name: no-name-9d552bc0-4a4a-4c7a-92ca-23d6561036e2


[2025-05-13 11:26:05,467: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/DikshantPatel2210/KidneyDiseaseClassification-DLProject "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "DikshantPatel2210/KidneyDiseaseClassification-DLProject"

[2025-05-13 11:26:05,474: INFO: helpers: Initialized MLflow to track repo "DikshantPatel2210/KidneyDiseaseClassification-DLProject"]


...

Initialized MLflow to track repo "DikshantPatel2210/KidneyDiseaseClassification-DLProject"

[2025-05-13 11:29:38,460: INFO: helpers: Initialized MLflow to track repo "DikshantPatel2210/KidneyDiseaseClassification-DLProject"]


Repository DikshantPatel2210/KidneyDiseaseClassification-DLProject initialized!

[2025-05-13 11:29:38,462: INFO: helpers: Repository DikshantPatel2210/KidneyDiseaseClassification-DLProject initialized!]
Epoch 1/3
8/8 [==============================] - ETA: 0s - loss: 1.7887 - accuracy: 0.3164[2025-05-13 11:29:57,790: INFO: callback: [Epoch 001] Custom Objective = 0.005676,
 Train Acc = 0.3164, Train Loss = 1.7887, Val Acc = 0.4077, Val Loss = 1.3867]

Epoch 00001: val_objective did not improve from 0.38744
8/8 [==============================] - 7s 839ms/step - loss: 1.7887 - accuracy: 0.3164 - val_loss: 1.3867 - val_accuracy: 0.4077 - val_objective: 0.0057 - lr: 6.1821e-04
Epoch 2/3
7/8 [=========================>....] - ETA: 0s - loss: 1.6567 - accuracy: 0.3527[2025-05-13 11:30:02,046: INFO: callback: [Epoch 002] Custom Objective = 0.195316,
 Train Acc = 0.3633, Train Loss = 1.5988, Val Acc = 0.4077, Val Loss = 1.3864]

Epoch 00002: val_objective did not improve from 0.38744
8/8 [==============================] - 4s 594ms/step - loss: 1.5988 - accuracy: 0.3633 - v

[I 2025-05-13 11:30:35,161] Trial 4 finished with value: 0.3306620419025421 and parameters: {'n_conv_layers': 6, 'n_dense_layers': 2, 'optimizer': 'sgd', 'Conv2D_strides_size': '2x2', 'Conv2D_kernel_size': '3x3', 'MaxPooling2D_strides_size': '2x2', 'MaxPooling2D_kernel_size': '2x2', 'learning_rate': 0.0006182110838061356, 'filters0': 128, 'filters1': 128, 'filters2': 64, 'filters3': 64, 'filters4': 512, 'filters5': 512, 'dense_units0': 32, 'dense_units1': 512}. Best is trial 0 with value: 0.3697035610675812.


In [15]:
 class OptunaModelTunnerSimple:
    def __init__(self, training_set, validation_set, train_df, class_weights_dict, class_names, callbacks):
        self.training_set = training_set
        self.validation_set = validation_set
        self.train_df = train_df
        self.class_weights_dict = class_weights_dict
        self.class_names = class_names
        self.callbacks = callbacks
    def create_model(self, trial):
        model = Sequential()
        model.add(Conv2D(32, (3, 3), activation="relu", padding="same", input_shape=(224, 224, 1)))
        model.add(BatchNormalization())
        model.add(MaxPooling2D((2, 2)))

        for i in range(trial.suggest_int("n_conv", 1, 3)):
            filters = trial.suggest_categorical(f"filters_{i}", [32, 64, 128])
            model.add(Conv2D(filters, (3, 3), activation="relu", padding="same"))
            model.add(BatchNormalization())
            model.add(MaxPooling2D((2, 2)))

        model.add(GlobalAveragePooling2D())

        for i in range(trial.suggest_int("n_dense", 1, 2)):
            units = trial.suggest_categorical(f"dense_units_{i}", [64, 128, 256])
            model.add(Dense(units, activation="relu"))
            model.add(Dropout(0.3))

        model.add(Dense(4, activation="softmax"))

        optimizer_choice = trial.suggest_categorical("optimizer", ["adam", "sgd", "RMSprop"])
        lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)

        if optimizer_choice == "adam":
            optimizer = Adam(learning_rate=lr)
        elif optimizer_choice == "sgd":
            optimizer = SGD(learning_rate=lr, momentum=0.9)
        else:
            optimizer = RMSprop(learning_rate=lr)

        model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
        return model

    def objective(self, trial):
        dagshub.init(repo_owner='DikshantPatel2210', repo_name='KidneyDiseaseClassification-DLProject', mlflow=True)
        mlflow.set_tracking_uri("https://dagshub.com/DikshantPatel2210/KidneyDiseaseClassification-DLProject.mlflow")
        mlflow.set_experiment("CNN_Optuna_MLFLOW")

        if mlflow.active_run():
            mlflow.end_run()

        with mlflow.start_run(run_name=f"trial_{trial.number}"):
            model = self.create_model(trial)

            mlflow.log_params(trial.params)
            mlflow.log_param("batch_size", 32)

            history = model.fit(
                self.training_set,
                steps_per_epoch=len(self.train_df) // 32,
                validation_data=self.validation_set,
                epochs=5,
                callbacks= self.callbacks,
                verbose=0
            )

            val_acc = max(history.history["val_accuracy"])
            train_loss = min(history.history["loss"])
            val_loss = min(history.history["val_loss"])
            score = val_acc - abs(train_loss - val_loss)

            mlflow.log_metrics({
                "train_accuracy": max(history.history["accuracy"]),
                "val_accuracy": val_acc,
                "train_loss": train_loss,
                "val_loss": val_loss,
                "score": score
            })

            # Log model
            input_example = np.random.rand(1, 224, 224, 1).astype(np.float32)
            output_example = model.predict(input_example)
            mlflow.tensorflow.log_model(model, "model", signature=infer_signature(input_example, output_example))

            K.clear_session()
            del model
            gc.collect()

            return score

In [12]:
class OptunaModelTunner12:
    def __init__(self,params: OptunaConfig, training_set, validation_set, train_df, callbacks, class_weights_dict, class_names):
        self.params = params
        self.training_set = training_set
        self.validation_set = validation_set
        self.train_df = train_df
        self.callbacks = callbacks
        self.class_weights_dict = class_weights_dict
        self.class_names = class_names
        
    def create_model(self, trial):
        
        n_conv_layers = trial.suggest_int("n_conv_layers", self.params.min_n_conv_layers, self.params.max_n_conv_layers)
        n_dense_layers = trial.suggest_int("n_dense_layers", self.params.min_n_dense_layers, self.params.max_n_dense_layers)
        optimizer_name = trial.suggest_categorical('optimizer', self.params.optimizer)
        #strides_size = trial.suggest_categorical("strides_size", self.params.strides_size)
        #filters = trial.suggest_categorical("filters", self.params.filters)
        #dense_units = trial.suggest_categorical("dense_units", self.params.dense_units)
        #lr = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
        
        
        
        
        
        
        model = Sequential()
        model.add(Conv2D(32, (3,3), strides=(2, 2), activation="relu", padding='same', input_shape=(224,224,1), kernel_initializer='he_normal'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D((2, 2), strides=(2, 2), padding='same'))
        
        for i in range(n_conv_layers):
            filters = trial.suggest_categorical(f"filters{i}", self.params.filters)
            model.add(Conv2D(filters, (3,3), strides=(2, 2), activation="relu", padding='same'))
            model.add(BatchNormalization())
            model.add(MaxPooling2D((2, 2), strides=(2, 2), padding='same'))
            
        model.add(GlobalAveragePooling2D())
        
        for i in range(n_dense_layers):
            dense_units = trial.suggest_categorical(f"dense_units{i}",self.params.dense_units)
            model.add(Dense(dense_units, activation='relu'))
            model.add(BatchNormalization())
            model.add(Dropout(0.3))
        
        model.add(Dense(4, activation='softmax'))
        lr = trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True)
        if optimizer_name == 'adam':
             
             optimizer = Adam(learning_rate=lr)
        elif optimizer_name == 'sgd':
             optimizer = SGD(learning_rate=lr)
        elif optimizer_name == "RMSprop":
             optimizer = RMSprop(learning_rate=lr)
        else:
             raise ValueError("Choose 'adam' ,'sgd' or 'RMSprop' for optimizer_name")

        model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=["accuracy"])

        return model
    
    def objective(self, trial):
        dagshub.init(repo_owner='DikshantPatel2210', repo_name='KidneyDiseaseClassification-DLProject', mlflow=True)
        mlflow.set_tracking_uri("https://dagshub.com/DikshantPatel2210/KidneyDiseaseClassification-DLProject.mlflow")
        mlflow.set_experiment("CNN_Optuna_MLFLOW")
        try:
            if mlflow.active_run():
               mlflow.end_run()

            input_example = np.random.rand(32, 224, 224, 1).astype(np.float32)
            output_example = np.random.rand(32, 4).astype(np.float32)

            with mlflow.start_run(run_name=f"trial_{trial.number}"):
                model = self.create_model(trial)

                # Log hyperparameters
                for param_name, param_value in trial.params.items():
                    mlflow.log_param(param_name, param_value)

                history = model.fit(
                    self.training_set,
                    steps_per_epoch=len(self.train_df) // 32,
                    validation_data=self.validation_set,
                    epochs=30,
                    callbacks=self.callbacks,
                    class_weight=self.class_weights_dict,
                    verbose=0
                )

                # Static params
                mlflow.log_param("image_size", "224x224x1")
                mlflow.log_param("Batchsize", 32)
                mlflow.log_param("loss function", "categorical_crossentropy")

                acc_val = history.history['val_accuracy'][-1]
                loss_train = history.history['loss'][-1]
                loss_val = history.history['val_loss'][-1]
                loss_diff = abs(loss_train - loss_val)
                objective_value = acc_val - loss_diff

                # Metrics
                mlflow.log_metric("train_accuracy", max(history.history['accuracy']))
                mlflow.log_metric("train_loss", min(history.history['loss']))
                mlflow.log_metric("val_accuracy", max(history.history['val_accuracy']))
                mlflow.log_metric("val_loss", min(history.history['val_loss']))
                mlflow.log_metric("acc_val - loss_diff", objective_value)

                # Predictions
             #   y_pred_probs = model.predict(self.validation_set)
             #   y_pred = np.argmax(y_pred_probs, axis=1)
             #   y_true = self.validation_set.classes

             #   report = classification_report(y_true, y_pred, target_names=self.class_names, output_dict=True)
             #   df_report = pd.DataFrame(report).transpose()
             #   report_path = f"classification_report_trial_{trial.number}.csv"
             #   df_report.to_csv(report_path)
            #   mlflow.log_artifact(report_path)

                # Confusion Matrix
            #    cm = confusion_matrix(y_true, y_pred)
              #  fig, ax = plt.subplots(figsize=(8, 6))
             #   sns.heatmap(cm, annot=True, fmt='d', xticklabels=self.class_names, yticklabels=self.class_names, cmap='Blues', ax=ax)
             #   plt.xlabel('Predicted')
             #   plt.ylabel('Actual')
             #   plt.title('Confusion Matrix')
             #   fig_path = f"conf_matrix_trial_{trial.number}.png"
             #   fig.savefig(fig_path)
             #   mlflow.log_artifact(fig_path)
             #   plt.close(fig)

                # Log model
                signature = infer_signature(input_example, output_example)
                mlflow.tensorflow.log_model(model, artifact_path="model", signature=signature)

                return objective_value

        except Exception as e:
            print(f"[Trial Failed] Error: {e}")
            mlflow.log_param("failed_trial", True)
            mlflow.log_param("error_msg", str(e)[:500])
            return float("nan")

        finally:
            mlflow.end_run()
            try:
                del model
            except:
                pass
            K.clear_session()
            gc.collect()